In [57]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine



In [3]:
suicide_file = "Resources/master.csv"
happiness_2015 = "Resources/2015.csv"
happiness_2016 = "Resources/2016.csv"
happiness_2017 = "Resources/2017.csv"

In [7]:
suicide_df = pd.read_csv(suicide_file)

hap_2015 = pd.read_csv(happiness_2015)
hap_2016 = pd.read_csv(happiness_2016)
hap_2017 = pd.read_csv(happiness_2017)

hap_2015["year"] = 2015
hap_2016["year"] = 2016
hap_2017["year"] = 2017

hap_2015 = hap_2015[["Country", "Happiness Rank", "Happiness Score", "Economy (GDP per Capita)", \
                    "Family", 'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)',\
                     'Generosity', 'Dystopia Residual', 'year']]

hap_2016 = hap_2016[["Country", "Happiness Rank", "Happiness Score", "Economy (GDP per Capita)", \
                    "Family", 'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Dystopia Residual', \
                    'year']]

hap_2017 = hap_2017[["Country", "Happiness.Rank", "Happiness.Score", "Economy..GDP.per.Capita.", \
                    "Family", 'Trust..Government.Corruption.', 'Generosity', 'Dystopia.Residual', \
                    'year']]

hap_2015 = hap_2015.rename(columns={'Country':'country',"Happiness Rank":"happiness_rank", \
                                    "Happiness Score":"happiness", \
                                    "Economy (GDP per Capita)":"economy", "Family": "family", \
                                    'Health (Life Expectancy)':'health', \
                                    'Freedom': 'freedom', 'Trust (Government Corruption)':'trust' ,\
                                    'Generosity':'generosity', 'Dystopia Residual':'dystopia'})

hap_2016 = hap_2016.rename(columns={'Country':'country',"Happiness Rank":"happiness_rank", "Happiness Score":"happiness", \
                                    "Economy (GDP per Capita)":"economy", "Family": "family", \
                                    'Health (Life Expectancy)':'health', \
                                    'Freedom': 'freedom', 'Trust (Government Corruption)':'trust' ,\
                                    'Generosity':'generosity', 'Dystopia Residual':'dystopia'})

hap_2017 = hap_2017.rename(columns={'Country':'country',"Happiness.Rank":"happiness_rank", "Happiness.Score":"happiness", \
                                    "Economy..GDP.per.Capita.":"economy", "Family": "family", \
                                    'Health..Life.Expectancy.':'health', \
                                    'Freedom': 'freedom', 'Trust..Government.Corruption.':'trust' ,\
                                    'Generosity':'generosity', 'Dystopia.Residual':'dystopia'})


happiness_df = hap_2015.append(hap_2016)
happiness_df = happiness_df.append(hap_2017)

In [40]:
happiness_df.head()

,country,dystopia,economy,family,freedom,generosity,happiness,happiness_rank,health,trust,year
0,Switzerland,2.51738,1.39651,1.34951,0.66557,0.29678,7.587,1,0.94143,0.41978,2015
1,Iceland,2.70201,1.30232,1.40223,0.62877,0.43630,7.561,2,0.94784,0.14145,2015
2,Denmark,2.49204,1.32548,1.36058,0.64938,0.34139,7.527,3,0.87464,0.48357,2015
3,Norway,2.46531,1.45900,1.33095,0.66973,0.34699,7.522,4,0.88521,0.36503,2015
4,Canada,2.45176,1.32629,1.32261,0.63297,0.45811,7.427,5,0.90563,0.32957,2015


In [62]:
countries = happiness_df['country'].unique()
countries.tolist()
country_lookup = pd.DataFrame(countries, columns=['country_name'])

country_id = []
for i in range(len(country_lookup)):
    country_id.append(i+1)

country_lookup['country_id'] = country_id
country_lookup.head()

,country_name,country_id
0,Switzerland,1
1,Iceland,2
2,Denmark,3
3,Norway,4
4,Canada,5


In [41]:
suicide_df.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [56]:
group = suicide_df.groupby(['country'])
suicides_2015 = group.apply(lambda x: x[x['year'] == 2015]['suicides_no'].sum() )
suicides_2016 = group.apply(lambda x: x[x['year'] == 2016]['suicides_no'].sum() )

suicides_2015

country
Albania                             0
Antigua and Barbuda                 1
Argentina                        3073
Armenia                            74
Aruba                               0
Australia                        3027
Austria                          1251
Azerbaijan                          0
Bahamas                             0
Bahrain                             0
Barbados                            0
Belarus                             0
Belgium                          1867
Belize                             26
Bosnia and Herzegovina              0
Brazil                          11163
Bulgaria                            0
Cabo Verde                          0
Canada                              0
Chile                            1838
Colombia                         2332
Costa Rica                          0
Croatia                           739
Cuba                             1511
Cyprus                             40
Czech Republic                   1387
Denm

In [58]:
connection_string = "postgres:postgres@localhost:5432/etl_db"
engine = create_engine(f'postgresql://{connection_string}')

In [59]:
# need to create etl_db to work

engine.table_names()

OperationalError: (psycopg2.OperationalError) FATAL:  database "customer_db" does not exist

(Background on this error at: http://sqlalche.me/e/e3q8)